Working with the Titanic Data sets off Kaggle
https://www.kaggle.com/c/titanic

Authors: C.W. and Jacob D.
Started: 9/4/2020

In [2]:
# === Initialize the needed modules ===
import sys 
print("Python version: {}". format(sys.version))

# -- for data processing -- 
import numpy as np #foundational package for scientific computing
print("NumPy version: {}". format(np.__version__))

import pandas as pd #collection of functions for data processing and analysis
print("pandas version: {}". format(pd.__version__))

# -- for Data Visualization -- 
import matplotlib.pyplot as plt #collection of functions for scientific visualization
print("matplotlib version: {}". format(matplotlib.__version__))
import seaborn as sns

# -- for Mathematics and Machine Learning Algorithms -- 
import scipy as sp #collection of functions for scientific computing and advance mathematics
print("SciPy version: {}". format(sp.__version__)) 

import sklearn #collection of machine learning algorithms
print("scikit-learn version: {}". format(sklearn.__version__))

# -- for misc uses -- 
import random
import time

print("=== Loaded Modules ===")

Python version: 3.7.6 (default, Jan  8 2020, 20:23:39) [MSC v.1916 64 bit (AMD64)]
NumPy version: 1.18.1
pandas version: 1.0.1
matplotlib version: 3.1.3
SciPy version: 1.4.1
scikit-learn version: 0.22.1
=== Loaded Modules ===


In [15]:
# === Load the Data == 
data_pd_train = pd.read_csv('train.csv')
data_pd_test  = pd.read_csv('test.csv')
print("=== Loaded Data ===")

data_np_train = data_pd_train.to_numpy()
print('Data size is ',data_np_train.shape) # need data in Numpy format
print("-"*10)
print("=== Data Information ===")
print(data_pd_train.info()) #need data in Panda format
print("-"*10)
print("=== Data Header and a little data ===")
data_pd_train.head()  #need data in Panda format

=== Loaded Data ===
Data size is  (891, 12)
----------
=== Data Information ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None
----------
=== Data Header and a little data ===


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [10]:
# === Identify Missing Data === 
print('Train columns with null values:\n', data_pd_train.isnull().sum())
print("-"*10)

print('Test/Validation columns with null values:\n', data_pd_test.isnull().sum())
print("-"*10)


Train columns with null values:
 PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
----------
Test/Validation columns with null values:
 PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64
----------
